# Balance Dataset with full Inner-anotater Agreement
## This notebook balances a dataset by minority class label along with only including data where all taggers have agreed on a tag.
### Note: This is primarily designed for the csv output from the json_parser.ipynb file.

### Import the required libraries:

In [ ]:
import pandas as pd
import csv

### (OPTIONAL): If using google colabs, mount your drive so you can reference a file system to reading in and storing datasets.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

### Indicate the csv file that you want to read in:
#### Note: Make sure that the data, label, and other dataset column names match the dataset of interest.

In [ ]:
file_in = "/content/gdrive/My Drive/Colab Notebooks/various_json/localize/new_localize_data_parsed.csv"
data_col = "REVIEW"
label_col = "TAG"
df = pd.read_csv(file_in, engine = 'python');
df.sort_values(by=['UPDT_TIME'], inplace=True, ascending=False) # Sort by timestamp
df = df.reset_index() # remove later
print("Number of samples:", len(df))
df.head(5)

### This function determines the count of each class type for writing a balanced file.

In [ ]:
def num_balanced_labels(data_col, label_col, df):
    com_list = []
    num_0 = 0
    num_1 = 0
    print("Balance: Total observations to parse:", len(df))
    for i in range(len(df)):
        if i%2000 == 0:
            print("Update:", i)
        sen = df[data_col][i]
        val = int(df[label_col][i])
        if sen == "" or isinstance(sen, str) == False or sen == " ":
                continue
        sen = sen.strip()
        if sen in com_list:
            continue
        com_list.append(sen)
        if val == 1:
            num_1 = num_1 + 1
        else:
            num_0 = num_0 + 1
    num_labels = min(num_0, num_1)
    com_list = []
    print("The number of observations of class False is:", num_0)
    print("The number of observations of class True is:", num_1)
    return(num_labels)

### The code below creates the new cleaned dataset:

In [ ]:
num_labels = num_balanced_labels(data_col, label_col, df)
ls = []
lstag = []
idt = []
num_0 = 0 # Negative class label
num_1 = 0 # Positive class label
dropthese = [] # Indexes of comments to be dropped (comments without full inner-annotator agreement)
num_same = 0 # Number of times where a duplicate (annotated by multiple people) comment was tagged the same.
num_different = 0 # Number of times where a duplicate (annotated by multiple people) comment was not tagged the same.
print("Create: Balance observations")
for i in range(len(df)):
    if i%2000 == 0:
        print("Update:", i) # Tracks progress
    text = df[data_col][i]
    if text == "" or isinstance(text, str) == False or text == " ":
        continue
    text = text.strip()
    if text in ls:
        for j in range(len(idt)):
            if text == ls[j]:
                if df["TAG"][idt[j]] == df["TAG"][i]:
                    num_same = num_same + 1
                else:
                    num_different = num_different + 1
                    if df["TAGGER"][idt[j]] == df["TAGGER"][i]: # If tagger is the same, that means they are redoing a tag, so we use the most recent label.
                        upexis = df["UPDT_TIME"][idt[j]]
                        upcur = df["UPDT_TIME"][i]
                        if (upexis < upcur):
                            lstag[j] = df["TAG"][i]
                            newtag = df["TAG"][i] # change label balance counts
                            if newtag == 0:
                                num_1 = num_1 - 1
                                num_0 = num_0 + 1
                            else:
                                num_1 = num_1 + 1
                                num_0 = num_0 - 1
                    else: # If tagger is different, don't include this comment in the dataset.
                        dropthese[j] = 1
        continue
    val = int(df[label_col][i])
    if val == 1:
        num_1 = num_1 + 1
    else:
        num_0 = num_0 + 1
        val = 0 # Make sure negative label is 0 (ML libraries don't handle negative labels well)
    if val == 0 and (num_0 > num_labels):
        continue
    if val == 1 and (num_1 > num_labels):
        continue
    ls.append(text)
    lstag.append(val)
    idt.append(i)
    dropthese.append(0)
print("Number of times duplicate comments were tagged the same:", num_same)
print("Number of times duplicate comments were not tagged the same:", num_different)

### Indicate the name of the output balanced csv file:

In [ ]:
file_out = "/content/gdrive/My Drive/Colab Notebooks/various_json/localize/new_localize_data_balanced.csv"

### Create the new csv file with class balanced data.

In [ ]:
with open(file_out, 'w', newline = '') as f:
    writer = csv.writer(f)
    writer.writerow([label_col, data_col])
    for i in range(len(ls)):
        if dropthese[i] == 1: # Ignore the comments that were previously determined to not have full inner-annotator agreement.
            continue
        val = lstag[i]
        text = ls[i]
        writer.writerow([val, text])